In [58]:
import pandas as pd
import json

In [57]:
file1_df = pd.read_json('raw_data/purchase_data.json')
file2_df = pd.read_json('raw_data/purchase_data2.json')
print (len(file1_df))
print (len(file2_df))
#pymoli_df = pd.merge(file1_df,file2_df, on="", how="inner")

780
78


In [66]:
#Check to make sure columns are the same between both files
file1_df.columns,file2_df.columns

(Index(['Age', 'Gender', 'Item ID', 'Item Name', 'Price', 'SN'], dtype='object'),
 Index(['Age', 'Gender', 'Item ID', 'Item Name', 'Price', 'SN'], dtype='object'))

In [67]:
pymoli_df = file1_df.append(file2_df)

In [79]:
#player count
playerCount = len(pymoli_df['SN'])     #alternate way: playerCount = pymoli_df['SN'].count().mean()

In [115]:
#purchase analysis
pymoli_df['Item ID'].unique()


array([165, 119, 174,  92,  63,  10, 153, 169, 118,  99,  57,  47,  81,
        77,  44,  96, 123,  59,  91, 177,  78,   3,  11, 183,  65, 132,
       106,  49,  45, 155,  37,  48,  90,  13, 171,  25,   7, 124,  68,
        85, 120,  17, 141,  73, 151,  32,  51, 101, 140,  31,  34,   2,
        86,  39,  28, 160, 134,  83,  38, 158, 110, 122,  54, 105,  87,
        23, 144, 128, 175,  46, 150, 152, 108, 172, 167, 181,  20, 130,
       111, 103,  30, 139, 173,  55, 115,  35,  42,   9,  84, 180, 102,
        53,  18,  74, 126,  50,  62, 125, 121, 129, 149,  12,  71,  14,
        58,  27,  52,  66, 100, 112,  24,  94, 107,   0, 182,  97,  70,
        89,   1, 170,  93, 179,  36,  75, 143, 137, 176, 148, 127, 147,
       161, 154, 157, 116,  61, 131,  41, 145,  60, 162, 135,   8,  40,
        15,  29,  72, 114, 117,  79,  88, 104,  95,  64,  98,  33,  76,
       146, 166,  56,  22,  21,  16,  67, 133,  69, 159,  82, 113, 164,
         6, 163,   5,  19, 168, 136,  80,  26, 142, 178, 156, 10